# Agents for Amazon Bedrock - Associate Knowledge Base to Agent

This notebook provides sample code for associating a Knowledge Base for Amazon Bedrock to an existent Agent for Amazon Bedrock that has an Action Group attached to it.

### Use Case
We will update our restaurant assistant to allows customers to also ask questions about the restaurant menus. The architecture looks as following:

![Agent Architecture](./images/agent-architecture.png)

### Notebook Walkthrough

In this notebook we will:
- Retrieve the saved variables from the previous notebook
- Create a Knowledge Base and its pre-requirements (including OpenSearch Servelless Collection and Indexes)
- Syncronize Knowledge Base with documents containing restaurant menus
- Update Agent IAM role to allow for Knowledge Base access
- Associate Knowledge Base with Restaurant Agent
- Test Agent invocation with Knowledge Base access


### Next Steps: 
In the next lab, we will test the agent invocation with Action Group and Knowledge Base requests as well as provide extra information to the agent using Prompt attributes

### Pre-requisites

Before starting this lab, we need to load the variables that we stored in the previous notebook.

In [ ]:
%store -r

In [ ]:
agent_id

Let's now import the necessary libraries and initiate the required boto3 clients

In [ ]:
from knowledge_base import BedrockKnowledgeBase
import boto3
import os
import time
import json

In [ ]:
iam_client = boto3.client('iam')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
s3_client = boto3.client('s3')

### Setting up Knowledge Base Information

We will now set the variables that define our Knowledge Base:

- **knowledge_base_name**: provides the name of the Knowledge Base to be created, in this case `booking-agent-kb`
- **knowledge_base_description**: the description of the knowledge base used to display the agents list on the console. This description is **not** part of the agent's prompts
- **bucket_name**: name of the bucket containing the Knowledge Base documents

In [ ]:
knowledge_base_name = f'{agent_name}-kb'
knowledge_base_description = "Knowledge Base containing the restaurant menu's collection"
bucket_name = f'{agent_name}-{suffix}'

### Creating Knowledge Base for Amazon Bedrock

We will now going to create a Knowledge Base for Amazon Bedrock and its requirements including:
- [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/) for the vector database
- [AWS IAM](https://aws.amazon.com/iam/) roles and permissions
- [Amazon S3](https://aws.amazon.com/s3/) bucket to store the knowledge base documents

To create the knowledge base and its dependencies, we will use the `BedrockKnowledgeBase` support class, available in this folder. It allows you to create a new knowledge base, ingest documents to the knowledge base data source and delete the resources after you are done working with this lab

In [ ]:
knowledge_base = BedrockKnowledgeBase(
    kb_name=knowledge_base_name,
    kb_description=knowledge_base_description,
    data_bucket_name=bucket_name
)

We now upload the knowledge base documents to S3

In [ ]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("kb_documents", bucket_name)

And ingest the documents to the knowledge base

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base.start_ingestion_job()

### Testing Knowledge Base

Let's now test that the created knowledge base works as expected. To do so, we first retrieve the knowledge base id

In [ ]:
kb_id = knowledge_base.get_knowledge_base_id()

Next we can use the [`RetrieveAndGenerate`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) API from boto3 to retrieve the context for the question from the knowledge base and generate the final response

In [ ]:
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": "Which are the 5 mains available in the childrens menu?"
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, agent_foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

### Updating Agent role to allow Knowledge Base Retrieve and Retrieve and Generate queries

Now that our Knowledge Base is working, we will associate it with the agent. To do so, we first need to update the agent role to allow for retrieval from context from our knowledge base

In [ ]:
kb_policies_statements = [
    {
        "Sid": "QueryKB",
        "Effect": "Allow",
        "Action": [
            "bedrock:Retrieve",
            "bedrock:RetrieveAndGenerate"
        ],
        "Resource": [
            f"arn:aws:bedrock:{region}:{account_id}:knowledge-base/{kb_id}"
        ]
    }
]
bedrock_agent_kb_policy_statement = {
    "Version": "2012-10-17",
    "Statement": kb_policies_statements
}
bedrock_agent_kb_policy_json = json.dumps(bedrock_agent_kb_policy_statement)
kb_policy_name = f"{agent_name}-kb-{kb_id}"
agent_kb_policy = iam_client.create_policy(
    PolicyName=kb_policy_name,
    PolicyDocument=bedrock_agent_kb_policy_json
)
iam_client.attach_role_policy(
    RoleName=agent_role['Role']['RoleName'],
    PolicyArn=agent_kb_policy['Policy']['Arn']
)

#### Associate Knowledge Base with Agent

Finally, we can associate the new knowledge base with the agent using the [`AssociateAgentKnowledgeBase`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/associate_agent_knowledge_base.html) API from boto3

In [ ]:
time.sleep(30)
response = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description='Access the knowledge base when customers ask about the plates in the menu.',
    knowledgeBaseId=kb_id,
    knowledgeBaseState='ENABLED'
)

#### Preparing Agent

after updating our agent, we need to prepare it again to package all its new components

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
time.sleep(30)

### Invoking Agent

Now that our Agent has been updated, let's test it again. To do so we will again use the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) function from the boto3 Bedrock runtime client.

We will create the same support function called `invokeAgent` to allow us to invoke the agent with or without trace enabled and with or without session state. We will getinto more details about those concepts in the `03_invoke_agent.ipynb` notebook.

In [ ]:
def invokeAgent(query, session_id, enable_trace=False, session_state=dict()):
    end_session:bool = False
    
    # invoke the agent API
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session,
        sessionState=session_state
    )
    
    if enable_trace:
        logger.info(pprint.pprint(agentResponse))
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

Now we can test it by asking a question where the answer is available in the knowledge base documents

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = "What are the starters in the childrens menu?"
response = invokeAgent(query, session_id)
print(response)

### Next Steps

Before moving to the next notebook, let's store a couple of variables to continue working the the same notebook.

Next we will test our agent

In [ ]:
%store kb_id
%store knowledge_base_name
%store knowledge_base_description
%store kb_policy_name
%store bucket_name